In [28]:
from indicators import compute_ema, compute_macd, compute_log_return
import pandas as pd
from io import StringIO
import os

In [29]:
path = '../raw_data/'
BELEX15 = 'BELEX15.csv'
SOFIX = 'SOFIX.csv'
CBX10 = 'CBX10.csv'
SP500 = 'S&P500.csv'
SP600 = 'S&P600.csv'

The first step, of the analysis process, after haveing downloaded the data was to import the csv files of the 5 selected indexes and uniform their structure.
The standardization process differed from dataset to dataset but overall it can be summarized as:
1. import the csv file
2. fix the columns format
3. subset the columns to maintain only the necessary ones (date and price)
4. sort the data based on the date

In [30]:
#BELEX15 index
BELEX15 = pd.read_csv((path + BELEX15), delimiter=';')

BELEX15['date'] = pd.to_datetime(BELEX15['date'], format='%d.%m.%Y', errors='coerce')
BELEX15 = BELEX15[['date', 'Value']]
BELEX15['Value'] = BELEX15['Value'].str.replace('.', '').str.replace(',', '.').astype(float)
BELEX15 = BELEX15.rename(columns={'Value': 'price'})

# Sort DataFrame based on 'date' and reset the index
BELEX15 = BELEX15.sort_values(by='date').reset_index(drop=True)

BELEX15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4345 entries, 0 to 4344
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    4343 non-null   datetime64[ns]
 1   price   4343 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 68.0 KB


In [31]:
#SOFIX index
SOFIX = pd.read_csv((path + SOFIX), delimiter=';')

SOFIX['date'] = pd.to_datetime(SOFIX['date'], format='%Y-%m-%d', errors='coerce')
SOFIX = SOFIX.rename(columns={'Value': 'price'})

# Sort DataFrame based on 'date' and reset the index
SOFIX = SOFIX.sort_values(by='date').reset_index(drop=True)

SOFIX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4451 entries, 0 to 4450
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    4451 non-null   datetime64[ns]
 1   price   4451 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 69.7 KB


In [32]:
# CBX10 index
CBX10 = pd.read_csv((path + CBX10), delimiter=',')

CBX10['date'] = pd.to_datetime(CBX10['date'], format='%Y-%m-%d', errors='coerce')
CBX10 = CBX10[['date', 'last_value']]
CBX10 = CBX10.rename(columns={'last_value': 'price'})

# Sort DataFrame based on 'date' and reset the index
CBX10 = CBX10.sort_values(by='date').reset_index(drop=True)

CBX10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3240 entries, 0 to 3239
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3240 non-null   datetime64[ns]
 1   price   3239 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 50.8 KB


In [33]:
# SP500 index
SP500 = pd.read_csv((path + SP500), delimiter=',')

SP500['date'] = pd.to_datetime(SP500['date'], format='%m/%d/%Y')
SP500 = SP500[['date', 'Price']]
SP500['Price'] = SP500['Price'].replace('[\$,]', '', regex=True).astype(float)
SP500 = SP500.rename(columns={'Price': 'price'})

# Sort DataFrame based on 'date' and reset the index
SP500 = SP500.sort_values(by='date').reset_index(drop=True)

SP500.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4531 entries, 0 to 4530
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    4531 non-null   datetime64[ns]
 1   price   4531 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 70.9 KB


In [34]:
# SP600 index
SP600 = pd.read_csv((path + SP600), delimiter=',')

SP600['Date'] = pd.to_datetime(SP600['Date'], format='%Y-%m-%d')
# Keep only 'date' and 'Close' columns
SP600 = SP600[['Date', 'Close']]
SP600['Close'] = SP600['Close'].round(2)
SP600 = SP600.rename(columns={'Close': 'price', 'Date': 'date'})

# Sort DataFrame based on 'date' and reset the index
SP600 = SP600.sort_values(by='date').reset_index(drop=True)

SP600.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3272 entries, 0 to 3271
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3272 non-null   datetime64[ns]
 1   price   3272 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 51.3 KB


what we obtain are the datasets, containing each index price at each day, for 5 indexes that vastly differ in price.

In [35]:
# Displaying the head of each DataFrame
print("BELEX15:")
print(BELEX15.head())
print("\nSOFIX:")
print(SOFIX.head())
print("\nCBX10:")
print(CBX10.head())
print("\nSP500:")
print(SP500.head())
print("\nSP600:")
print(SP600.head())

BELEX15:
        date    price
0 2005-10-04   998.50
1 2005-10-05  1009.26
2 2005-10-06  1019.67
3 2005-10-07  1028.04
4 2005-10-10  1034.93

SOFIX:
        date   price
0 2005-01-03  623.59
1 2005-01-04  625.86
2 2005-01-05  617.68
3 2005-01-06  624.09
4 2005-01-07  622.88

CBX10:
        date    price
0 2010-01-04  1068.63
1 2010-01-05  1082.83
2 2010-01-07  1089.46
3 2010-01-08  1097.11
4 2010-01-11  1114.68

SP500:
        date   price
0 2005-01-03  1202.1
1 2005-01-04  1188.0
2 2005-01-05  1183.7
3 2005-01-06  1187.9
4 2005-01-07  1186.2

SP600:
        date   price
0 2010-01-04  339.64
1 2010-01-05  338.48
2 2010-01-06  338.15
3 2010-01-07  340.39
4 2010-01-08  341.47


In [36]:
def filter_data_by_date_range(df, start_date, end_date):
    """
    Filter DataFrame rows based on a date range.

    Parameters:
    - df: pandas DataFrame
    - start_date: string or Timestamp
    - end_date: string or Timestamp

    Returns:
    - filtered DataFrame
    """
    # Convert start_date and end_date to Timestamp
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Filter DataFrame based on the date range
    filtered_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)].reset_index(drop=True)

    return filtered_df

We then subset the dataset to obtain only the period of interest ranging from [01 Jan 2010] to [31 Dec 2022] and indicators to be used during the modeling process.

In [37]:
BELEX15 = filter_data_by_date_range(BELEX15, '2010-01-01', '2022-12-31')
SOFIX = filter_data_by_date_range(SOFIX, '2010-01-01', '2022-12-31')
CBX10 = filter_data_by_date_range(CBX10, '2010-01-01', '2022-12-31')
SP600 = filter_data_by_date_range(SP600, '2010-01-01', '2022-12-31')
SP500 = filter_data_by_date_range(SP500, '2010-01-01', '2022-12-31')

In [38]:
# Creating indicators
from indicators import compute_ema, compute_macd, compute_log_return
for index in [BELEX15,SOFIX, CBX10, SP600, SP500]:
    index['EMA_10'] = compute_ema(index)
    index['MACD'] = compute_macd(index)
    index['Log_Return'] = compute_log_return(index)

In [39]:
SOFIX.to_csv('../data/SOFIX_.csv', index=False)
BELEX15.to_csv('../data/BELEX15_.csv', index=False)
CBX10.to_csv('../data/CBX10_.csv', index=False)
SP600.to_csv('../data/SP600_.csv', index=False)
SP500.to_csv('../data/SP500_.csv', index=False)